#### Creating client and accessing google sheet

In [ ]:
import pygsheets
import time
# Create the Client
# Enter the name of the downloaded KEYS file in service_account_file

client = pygsheets.authorize(service_account_file="credentials.json")
  
# Sample command to verify successful authorization of pygsheets

print(client.spreadsheet_titles()) # Prints the names of the spreadsheets shared with service account

In [ ]:
#user inputs needed to specify which spreadsheet you will be working on
url="https://docs.google.com/spreadsheets/d/1k7_Qq4JGoqyW_cR68GRvqBfoCQkOXKt0pZSXEJxpyNw/edit#gid=0"
spreadsheetID="1k7_Qq4JGoqyW_cR68GRvqBfoCQkOXKt0pZSXEJxpyNw"

In [ ]:
#open the particular spreadsheet
#spreadsht = client.open("Asco 2022 automation")
#spreadsht=client.open_by_url("https://docs.google.com/spreadsheets/d/1k7_Qq4JGoqyW_cR68GRvqBfoCQkOXKt0pZSXEJxpyNw/edit#gid=0")
spreadsht=client.open_by_url(url)
# opens a worksheet by its name/title
ws_rd = spreadsht.worksheet("title", "Raw data")

In [ ]:
#create copies of raw data to work on 
for i in range(2):
    ws_rd.copy_to(spreadsheetID)

In [ ]:
#open the copies and rename them to base planner and version_1 respectively
ws_v1 = spreadsht.worksheet("title", "Copy of Raw data")
ws=spreadsht.worksheet("title", "Copy of Raw data 1")
# rename the copies as version_1 and base planner
ws.title="Base Planner"
ws_v1.title="version_1"

#### Delete unnecessary columns
****syntax: delete_cols(index, number=1)****

In [ ]:
#empty columns at the end
ws.delete_cols(25,2)   
#delete track and subtrack
ws.delete_cols(8,2)

In [ ]:
#delete keyword verification flag
ws.delete_cols(10,1)

In [ ]:
#delete drugs column
ws.delete_cols(12,1)

In [ ]:
#delete drug classification column
ws.delete_cols(15,1)

### Formatting

#### Change dimensions of rows and columns

In [ ]:
#changing column width of all columns except for id to 150
ws.adjust_column_width(2,24,150)

#changing column width of abstract column to 600
ws.adjust_column_width(5,5,600)

#adjust row height
ws.adjust_row_height(1, end=10000, pixel_size=50)

#### Change headers section

In [ ]:
headers=['A1','B1','C1','D1','E1','F1','G1','H1','I1','J1','K1','L1','M1','N1','O1','P1','Q1','R1',
        'S1']

In [ ]:
#bold the headers
for i in headers:
    #print(i)
    ws.cell(i).set_text_format('bold', True)
    #fill blue color for headers
    ws.cell(i).color = (0,0,1,0) 
    #change font color to white
    ws.cell(i).set_text_format('foregroundColor',value=(1,1,1,0))

#### Apply Borders

In [ ]:
from pygsheets import DataRange
drange = DataRange(start='A1', end='U10000', worksheet=ws)
drange.update_borders(top=True, right=True, bottom=True, left=True, inner_horizontal=True, inner_vertical=True, style='SOLID', width=1, red=0, green=0, blue=0)

#### Making copies for each entity QC

In [ ]:
#create 4 copies of the base planner for drug,drug class,indication and firm nct
for i in range(4):
    ws.copy_to(spreadsheetID)

In [ ]:
# opens worksheets by its name/title
ws_drug = spreadsht.worksheet("title", "Copy of Base Planner")
ws_drug_class=spreadsht.worksheet("title", "Copy of Base Planner 1")
ws_indication=spreadsht.worksheet("title", "Copy of Base Planner 2")
ws_firm=spreadsht.worksheet("title", "Copy of Base Planner 3")

In [ ]:
# rename the copies as drug,drug_class,indication and firm/nct
ws_drug.title="Drugs"
ws_drug_class.title="Drug_class"
ws_indication.title="indication"
ws_firm.title="firm/nct"

In [ ]:
#Hide other columns from each sheet

#Hide other columns for drug
ws_drug.update_dimensions_visibility(2, end=11, dimension='COLUMNS', hidden=True)
ws_drug.update_dimensions_visibility(15, end=19, dimension='COLUMNS', hidden=True)

#Hide other columns for drug_class
ws_drug_class.update_dimensions_visibility(2, end=14, dimension='COLUMNS', hidden=True)
ws_drug_class.update_dimensions_visibility(16, end=19, dimension='COLUMNS', hidden=True)

#Hide other columns for indication
ws_indication.update_dimensions_visibility(2, end=16, dimension='COLUMNS', hidden=True)
ws_indication.update_dimensions_visibility(18, end=19, dimension='COLUMNS', hidden=True)

#Hide other columns for firm/nct
ws_firm.update_dimensions_visibility(2, end=9, dimension='COLUMNS', hidden=True)
ws_firm.update_dimensions_visibility(11, end=17, dimension='COLUMNS', hidden=True)

Send these 4 sheets to specific teams for QC and receive the Qc'd data in the same 4 sheets with internal_ids kept consistent across all 4 files

#### Vlookup Qc'd data into Base Planner

In [ ]:
#primary drugs 
list_1 = ws_drug.range('L2:L10000',returnas='matrix')
#secondary drugs 
list_2 = ws_drug.range('M2:M10000',returnas='matrix')
#comparator drugs 
list_3 = ws_drug.range('N2:N10000',returnas='matrix')
#moas  
list_4 = ws_drug_class.range('O2:O10000',returnas='matrix')

In [ ]:
#flatten the above list
l1 = [item for sublist in list_1 for item in sublist]
l2 = [item for sublist in list_2 for item in sublist]
l3 = [item for sublist in list_3 for item in sublist]
l4 = [item for sublist in list_4 for item in sublist]

#### Add Qc'd values to base planner sheet
****syntax: worksheet.update_col(index, values, row_offset=0))****

In [ ]:
# add Qc'd values to base planner sheet
ws.update_col(12,l1,row_offset=1)  # primary drug
ws.update_col(13,l2,row_offset=1)  # secondary drug
ws.update_col(14,l3,row_offset=1)  # comparator drug
ws.update_col(15,l4,row_offset=1)  # moas

time.sleep(2) #allow some time to reflect the changes in google sheet

#### Vlookup Qc'd data into sessions file 

In [ ]:
#vlookup in raw data sheet
ws_v1 = spreadsht.worksheet("title", "version_1")

In [ ]:
ws_v1.update_col(16,l1,row_offset=1)  # primary drug
ws_v1.update_col(17,l2,row_offset=1)  # secondary drug
ws_v1.update_col(18,l3,row_offset=1)  # comparator drug
ws_v1.update_col(20,l4,row_offset=1)  # moas
time.sleep(2)

In [ ]:
#Change verification flag to 1
flag_list1 = ws_v1.range('L2:L10000',returnas='matrix')
flag_1 = [item for sublist in flag_list1 for item in sublist]

for idx, item in enumerate(flag_1):
    if item =='0':
        flag_1[idx] = 1

        
ws_v1.update_col(12,flag_1,row_offset=1)
time.sleep(0.1)

In [ ]:
#download the file as version 2
ws_v1.export(filename='automation_v2', path='C:/Users/admin/Downloads')

#### Once the explicit verfication is done it should be reflected in the ferma portal.


*1.Go to ferma congress--> particular conference-->upload modified csv and upload the downloaded file i.e "automation_v2"*
2.click on implicit keywords---> download all_session_keywords_grouped as version_3

### Part 2

#### Implicit run

Import the downloaded version_3 into same spreadsheet url as above

In [ ]:
#open version_3
ws_v3=spreadsht.worksheet("title", "version_3")

#### Vllokup firm and indications into base planner and version 3

In [ ]:
#indication  
list_5 = ws_indication.range('Q2:Q10000',returnas='matrix')
#agencies  
list_6 = ws_firm.range('J2:J10000',returnas='matrix')
#Acronym 
list_7 = ws_firm.range('R2:R10000',returnas='matrix')
#NCT 
list_8 = ws_firm.range('S2:S10000',returnas='matrix')

In [ ]:
l5 = [item for sublist in list_5 for item in sublist]
l6 = [item for sublist in list_6 for item in sublist]
l7 = [item for sublist in list_7 for item in sublist]
l8 = [item for sublist in list_8 for item in sublist]

In [ ]:
#update qc'd data into base planner
ws.update_col(17,l5,row_offset=1)  # indication
ws.update_col(10,l6,row_offset=1)  # agencies
ws.update_col(18,l7,row_offset=1)  # acronym
ws.update_col(19,l8,row_offset=1)  # nct
time.sleep(2)

In [ ]:
#update Qc'd data into sessions file
ws_v3.update_col(22,l5,row_offset=1)  # indication
ws_v3.update_col(13,l6,row_offset=1)  # agencies
ws_v3.update_col(23,l7,row_offset=1)  # acronym
ws_v3.update_col(24,l8,row_offset=1)  # nct
time.sleep(2)

In [ ]:
#get moas from sessions_file as values to be updated in base planner 
moas_list = ws_v3.range('T2:T10000',returnas='matrix')
moas=[item for sublist in moas_list for item in sublist]
ws.update_col(15,moas,row_offset=1)
time.sleep(1)

In [ ]:
#Change verification flag instances from 1 to 2
flag_list2 = ws_v3.range('L2:L10000',returnas='matrix')
flag_2 = [item for sublist in flag_list2 for item in sublist]


for idx, item in enumerate(flag_2):
    if item =='1':
        flag_2[idx] = 2

#update the column        
ws_v3.update_col(12,flag_2,row_offset=1)

In [ ]:
#download this version as v4 
ws_v3.export(filename='automation_v4', path='C:/Users/admin/Downloads')

#### Go to ferma congress and upload the above downloaded file into uplaod modified csv--> run implicit keywords